## Diseño de filtros digitales FIR e IIR

##### Johany Carmona
###### Estudiante UdeA

Se desarrolla aplicación básica para diseñar filtros de tipo FIR: pasa-bajas, pasa-altas, pasa-banda, y rechaza-banda, a partir de la definición del filtro pasa-bajos ideal (función SINC) y usando métodos de enventanado para reducción del fenómeno de Gibbs.

Se diseña un filtro FIR a partir de la respuesta en frecuencia para la separación de dos muestras de audio del sonido de dos animales de especies distintas.

Se diseña un filtro IIR para la distorsión de dos audios sonoros que serán analizados.

En primer lugar se realizará un análisis en el dominio de la frecuencia para todas las muestras; posteriormente se realizará el diseño de los filtros FIR e IIR, sus condiciones de estabilidad y causalidad.

### Análisis de los insumos de entrada del filtro.

Se realiza un anális en frecuencia para dos muestras de sonidos de audio a 113 y 120 BPM.

In [ ]:
from IPython.display import Audio # para escuchar la senal
from scipy.io.wavfile import read # libreria para lectura de archivos de audio
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal

In [ ]:
Fs = 44100 #Hz #Sampling frequency designed filter
filenames = ['audio0.wav','audio1.wav']

Contenido digital Creative Commons Obtenidos de:

hxxps://freesound[.]org/people/strangehorizon/sounds/702411/

hxxps://freesound[.]org/people/Snapper4298/sounds/696106/

In [ ]:
def get_audio(song):
    file_audio=(song)
    fs, x = read(file_audio)
    if len(x.shape) == 2: #For Stereo Audios
        L = x[:,0]
        R = x[:,1]
        maximum_stereo_value = float(max(float(max(abs(L))) , float(max(abs(R)))))
        L=L/float(max(abs(L))) #Left side Stereo Audio
        R=R/float(max(abs(R))) #Right side Stereo Audio
        del (x)
        x = L + R
        x=x/maximum_stereo_value #normalizing signal
    elif len(x.shape) == 1: #For Mono Audios
        maximum_stereo_value = float(max(abs(x)))
        x=x/maximum_stereo_value #normalizing sig 
    return x, fs

#Time domain vectors
def load_audios(filenames, Fs = Fs):
    songs = []
    freq_songs = []
    for filename in filenames:
        x, fs = get_audio(filename)
        x = x / max(abs(x))
        signal_time = (len(x)/Fs)
        new_samples_number = int(signal_time*Fs)
        x = signal.resample(x, new_samples_number)
        songs.append(x)
        freq_songs.append(Fs)
    return songs, freq_songs

songs = []
freq_songs = []
time_songs = []
#songs, freq_songs, time_songs = load_audios(filenames)
songs, freq_songs = load_audios(filenames)

def deleting_residuals(t, x):
    #Deleting residual samples caused by the convolutions process.
    sizes = [len(t), len(x)]
    if sizes[1] - sizes[0] > 0: x = x[:-(sizes[1] - sizes[0])]
    elif sizes[0] - sizes[1] > 0: t = t[:-(sizes[0] - sizes[1])]
    return t, x


def plot_signal_spectrum(freq_SIGNAL, SIGNAL, freq_signal, lims, zoom = False, title = ''):
    plt.figure(figsize=(20,8))
    plt.grid()
    plt.plot(freq_SIGNAL,abs(SIGNAL)**2 / max(lims), label = 'Frecuencia de muestreo $F_s$=%0.1f kHz'%(freq_signal*10**-3))
    
    normalized_lims = [0.0 , 1.0] if not(zoom) else [0.0, float(min(lims)/max(lims))] #zoom config
    plt.ylim(normalized_lims)
    plt.xlabel('Frecuencia [Hz]',fontsize=18) # x Axis
    plt.ylabel('Amplitude $|C_k|^{2}$',fontsize=18) # Y Axis
    plt.title(title)
    plt.legend()
    plt.show()

def graph_audio(x, fs, t = None, title = 'Señal Audio Original'):
    if type(t)==type(None): t=np.arange(0, float(len(x))/fs, 1.0/fs)
    
    plt.figure(figsize=(20,8))
    plt.grid()

    t, x = deleting_residuals(t, x)
    
    plt.plot(t,x, label = 'Frecuencia de Muestreo $F_s$=%0.1f kHz\nNúmero de muestras: %i muestras'%(fs*10**-3, len(x)))
    plt.xlabel('Time',fontsize=18) # x Axis
    plt.ylabel('Amplitude',fontsize=18) # Y Axis
    plt.title(title)
    plt.legend()
    plt.show()

#Frequency domain FFT vectors
def make_fft_audios(songs, freq_songs):
    SONGS = []
    freq_SONGS = []
    songs_number = len (filenames) #number of songs to filter.
    for i in range(songs_number):
        SONGS.append(np.fft.fft(songs[i]))
        freq_SONGS.append(np.fft.fftfreq(SONGS[i].size)*freq_songs[i])
    return SONGS, freq_SONGS
    
SONGS = []
freq_SONGS = []

SONGS, freq_SONGS = make_fft_audios(songs, freq_songs)

#Reproduction song objects vector
push_songs=[]
for i, filename in enumerate(filenames):
    print("\n\n\n\n\n________________________________________________________________________________________________________")
    print("Audio %i; Filename:"%i, filename)
    push_songs.append(Audio(songs[i], rate=freq_songs[i]))
    
    y = songs[i]

    Y = np.fft.fft(y) #Getting frequency response of the filtered signal.
    freq_Y = np.fft.fftfreq(Y.size)*freq_songs[i]
    
    graph_audio(songs[i], freq_songs[i])
    display(push_songs[i])
    
    lims = [max(abs(SONGS[i])**2),max(abs(Y)**2)]
    plot_signal_spectrum(freq_SONGS[i], SONGS[i], freq_songs[i], lims, zoom = False)

del (i)
del (filename)
del (y)
del (Y)
del (freq_Y)

Podemos observar en los audios importados que su frecuencia de muestreo es de Fs = 44.1 kHz, por ende, la componente frecuencial máxima que poseen dichos audios será $f_{max}$ = 22.05 (Criterio de Nyquist).

De esta manera, observamos que cada audio a simple vista abarca componentes frecuencias que están entre los 0-5kHz, y 0-3.3kHz, respectivamente.

De esta manera, empezaremos con el diseño de un filtro FIR pasa-banda a partir de la respuesta del filtro pasa-bajas ideal, luego un filtro pasa-altas que nos permitirá observar las componentes de frecuencia altas de cada audio, y posteriormente procederemos con el diseño de un filtro IIR pasa-bajas que nos ayudará con el filtrado de las componentes graves de cada señal.

### Diseño de filtros FIR Tipo I (Orden Impar) A Partir de la Respuesta Ideal (SINC)

Para este primer caso, el diseño del filtro FIR será un pasa-banda con un nivel de Ripple menor al 0.1%, una banda de Roll-off de 500 Hz, una frecuencia de corte entre 2.8 y 3.8 kHz, respectivamente. Sin embargo, el código en mención permite elegir los grados de libertad por parte del usuario, para las frecuencias de corte, tipo de frecuencia de muestreo, ventanas para truncamiento, &c.

Parámetros:

In [ ]:
#Sampling frequency
fs = 44100 #Hz

#Cut-off frequency
fc = 3550 #Hz

#Cut-off frequency for pass and drop bass filter
fc2 = 3800 #Hz
fc1 = 2800 #Hz

#Maximum Ripple Percentage
zigma = 0.1 #%

#Transition Bandwith

delta_fm = 500 #Hz

#Filter type: 'l': low pass; 'h': high pass; 'p': band pass; 's': band stop;
filter_type = 'p'

Se normaliza la frecuencia de corte en radianes

In [ ]:
import numpy as np
theta_cut = 2*np.pi*fc/fs
print("theta_cut: %.3f"%theta_cut)

Se calcula el nivel del rizado máximo en decibeles

In [ ]:
zigma_dec = 20*np.log10(zigma/100)
print("El filtro debe cumplir con un rizado máximo de delta_dec <= %.3f"%zigma_dec)

windows_zigma_dec = {'rect':-21.0,'hann':-44.0,'hamm':-53.0,'black':-74.0}

window = None

for item in list(windows_zigma_dec.items()):
    if float(item[1]) <= zigma_dec:
        window = item[0]
        print("Ventana que cumple el requerimiento del rizado máximo: ", item[0], ": %.3f"%item[1])
        break

windows_badwith_pass = {'rect':-21.0,'hann':-44.0,'hamm':-53.0,'black':-74.0}

Se calcula el ancho de banda de transición requerido en radianes:

In [ ]:
delta_tetha_m = 2*np.pi*delta_fm/fs
print("delta_tetha_m: %.3f"%delta_tetha_m)

Se calcula el orden M del filtro:

In [ ]:
def get_next_odd_number_after_rounding(input_number):
    # Round the input number to the nearest integer
    rounded_number = round(input_number)
    # If the rounded number is even, add 1 to make it odd
    if rounded_number % 2 == 0:
        return rounded_number + 1
    else:
        # If the rounded number is odd, add 2 to get the next odd number
        return rounded_number

windows_filter_orders = {'rect':get_next_odd_number_after_rounding(2*fs/delta_fm - 1),'hann': get_next_odd_number_after_rounding(4*fs/delta_fm),'hamm': get_next_odd_number_after_rounding(4*fs/delta_fm),'black': get_next_odd_number_after_rounding(6*fs/delta_fm)}

M = windows_filter_orders[window]

print("Orden de filtro mínimo: M >= %i"%M)

Calculando la respuesta al impulso de la función Sinc (Anticausal aún) para el lado derecho. 

In [ ]:
import pandas as pd


if (fc2<fc1):
    temp = fc2
    fc2 = fc1
    fc1 = temp

n_left = np.arange(-(M-1)/2, 0, dtype=int)
n_zero = np.array([0])
n_right = np.arange(1, (M-1)/2 + 1, dtype=int)


if filter_type == 'l':
    wc = 2*np.pi*fc/fs
    h_zero = wc / np.pi
    h_right = np.sin(wc*n_right)/(np.pi*n_right)

elif filter_type == 'h':
    wc = 2*np.pi*fc/fs
    h_zero = 1 - wc / np.pi
    h_right = - np.sin(wc*n_right)/(np.pi*n_right)

elif filter_type == 'p':
    wc2 = 2*np.pi*fc2/fs
    wc1 = 2*np.pi*fc1/fs
    h_zero = wc2 / np.pi - wc1 / np.pi
    h_right = np.sin(wc2*n_right)/(np.pi*n_right) - np.sin(wc1*n_right)/(np.pi*n_right)

elif filter_type == 's':
    wc2 = 2*np.pi*fc2/fs
    wc1 = 2*np.pi*fc1/fs
    h_zero = 1 - (wc2 / np.pi - wc1 / np.pi)
    h_right = - (np.sin(wc2*n_right)/(np.pi*n_right) - np.sin(wc1*n_right)/(np.pi*n_right))

h = list(h_right)[::-1] +[h_zero] + list(h_right)

h = np.round(h, 5)

n = np.array(list(n_left) + list(n_zero) + list(n_right))

# Create a DataFrame using pandas
df = pd.DataFrame({'n': n[int((M-1)/2):], 'h_i': h[int((M-1)/2)::]})

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



# Display the DataFrame
print(df)

Calculando la respuesta al impulso para el lado izquierdo dado que por su simetría positiva, se cumplirá h(n) = h(-n). En este caso sería el mismo vector del lado derecho pero invertido.

In [ ]:
for i in n[int((M-1)/2 + 1):]:
    print("h_i(-%i) = h_i(%i)"%(i, i))

Luego debemos volver causal el filtro (filtro real con retardo a la salida) antes de truncar con la ventana seleccionada, es decir que desplazaremos a la derecha del filtro (M-1)/2 unidades, esto es h(n) = h(n + (M-1)/2).

In [ ]:
for i in n:
    print("h_i(%i) => h_i(%i)"%(i, i + (M-1)/2))

n = n+(M-1)/2

Podemos corroborar la misma relación de simetría para el caso causal, en la cual se deberá cumplir h(n) = h(N - n -1)

In [ ]:
for i in n[:int((M-1)/2) + 1]:
    print("h_i(%i) = h_i(%i)"%(i, M - i -1))

Finalmente, tenemos que los coeficientes de la función de transferencia (causal) son:

In [ ]:
# Create a DataFrame using pandas
df = pd.DataFrame({'n': n, 'h_i': h})

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



# Display the DataFrame
print(df)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Plot the function
plt.figure(figsize=(20, 8)) 
plt.stem(n, h, label='h_i(n)')

# Add grid
plt.grid(True)

# Add titles and labels
plt.title('Función de Transferencia h_i(n) (Causal)')
plt.xlabel('n (samples)')
plt.ylabel('h_i(n) (amplitude)')

# Add legend
plt.legend()

# Show the plot
plt.show()


Luego se hallan los valores correspondientes a la ventana elegida en el diseño. De acuerdo a la simetría, sólo será necesario hallar los valores entre 0 y (M-1)/2, dado que los restantes se pueden hallar a través de la relación h(n) = h(N - n -1).

In [ ]:
n_left_and_half = np.arange(0, (M-1)/2 + 1, dtype=int)


if window == 'rect':
    w_left_and_half = np.ones(int((M-1)/2+1))

elif window == 'hann':
    w_left_and_half = 0.5*(1-np.cos(2*np.pi*n_left_and_half/(M-1)))

elif window == 'hamm':
    w_left_and_half = 0.54 -0.46*np.cos(2*np.pi*n_left_and_half/(M-1))

elif window == 'black':
    w_left_and_half = 0.42 -0.5*np.cos(2*np.pi*n_left_and_half/(M-1)) + 0.08*np.cos(4*np.pi*n_left_and_half/(M-1))

w = list(w_left_and_half) + list(w_left_and_half)[:-1][::-1]

w = np.round(w, 5)

n = np.arange(0, M, dtype=int)

# Create a DataFrame using pandas
df = pd.DataFrame({'n': n[:int((M-1)/2) +1], 'w': w[:int((M-1)/2) +1]})

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



# Display the DataFrame
print(df)

Ahora, podemos obtener el resto de los coeficientes de la ventana por uso de simetría h(n) = h(N - n -1) mencionada anteriormente.

In [ ]:
for i in n[:int((M-1)/2) + 1]:
    print("w(%i) = w(%i)"%(i, M - i -1))

Finalmente, tenemos que los coeficientes de la ventana usada son:

In [ ]:
# Create a DataFrame using pandas
df = pd.DataFrame({'n': n, 'w': w})

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



# Display the DataFrame
print(df)

Por último, el último paso es truncar la ventana con la respuesta al impulso de la función Sinc, con el objetivo de obtener la verdadera (práctica) respuesta al impulso del filtro diseñado, es decir: h(n) = h_i(n) * w(n)

In [ ]:
# Create a DataFrame using pandas

h = h*w

h = np.round(h, 5)

df = pd.DataFrame({'n': n, 'h': h})

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



# Display the DataFrame
print(df)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Plot the function
plt.figure(figsize=(20, 8)) 
plt.stem(n, h, label='h(n)')

# Add grid
plt.grid(True)

# Add titles and labels
plt.title('Función de Transferencia h(n) (Respuesta al impulso del filtro)')
plt.xlabel('n (samples)')
plt.ylabel('h(n) (amplitude)')

# Add legend
plt.legend()

# Show the plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

audio = SONGS[0]
t=np.arange(0, float(len(audio))/fs, 1.0/fs)
#t, audio = deleting_residuals(t, audio)

output_audio = np.convolve(audio, h, mode='same')

# Compute the FFT
fft_result = np.fft.fft(output_audio)
frequencies = np.fft.fftfreq(len(fft_result), 1/fs)

# Plot the original signal
plt.subplot(2, 1, 1)
plt.plot(t, output_audio)
plt.title('Original Signal')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

# Plot the magnitude spectrum (FFT result)
plt.subplot(2, 1, 2)
plt.plot(frequencies, np.abs(fft_result))
plt.title('Magnitude Spectrum (FFT)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')

plt.tight_layout()
plt.show()


In [ ]:

for i, filename in enumerate(filenames):
    hn = h
    print("\n______________________________\n")
    print("Audio %i; Filename:"%i, filename)
    push_songs.append(Audio(songs[i], rate=freq_songs[i]))
    
    y = np.convolve(hn,songs[i], mode="full")#Filtering the audio signal throught the designed filter.
    
    Y = np.fft.fft(y) #Getting frequency response of the filtered signal.
    freq_Y = np.fft.fftfreq(Y.size)*freq_songs[i]
    
    graph_audio(songs[i], freq_songs[i])
    display(push_songs[i])
    
    graph_audio(y, freq_songs[i], title = "Señal de audio de salida en filtro IIR pasa-bajas orden 1")
    display(Audio(y, rate=freq_songs[i]))
    
    lims = [max(abs(SONGS[i])**2),max(abs(Y)**2)]
    newrange = np.arange(0, len(freq_SONGS[i]), 1)
    plot_signal_spectrum(freq_SONGS[i], SONGS[i], freq_songs[i], lims, title ='')
    plot_signal_spectrum(freq_Y, Y, freq_songs[i], lims, zoom = True, title ='')

del (i)
del (filename)
del (y)
del (Y)
del (freq_Y)

#### Filtro FIR A Partir de Respuesta en Frecuencia

Para el siguiente caso, el diseño del filtro FIR pasa-altas deseado tendrá una frecuencia de corte ideal de 3.3 kHz, que a simple vista podemos observar que sería la más apropiada ya que permite pasar gran parte de las componentes del sonido del segundo audio (audio1.wav), y atenuar la mayoría de componentes del primer audio. 

Sin embargo, dada la presencia de Roll-Off en un filtro real, es necesario definir un ancho de banda de transición, el cual para nuestro caso, si elegimos un ancho de banda de transición de 500 Hz, el cual NO estará centrado en la frecuencia de corte original (fc ideal = 3.3kHz), sino en una nueva frecuencia de corte requerida (fc caso práctico = 3.5 kHz, de esta manera estaremos centrando todo el ancho de banda de transici´on de manera que NO permitamos el paso de componentes de frecuencia menores a < 3.3 kHz. Lo anterior se puede resumir de la siguiente manera $f_c -> f_c = f_c + delta_{fc} / 2$

Para el segundo caso, en el diseño del filtro IIR pasa-bajas se iterarán varias propiedades del período T con la finalidad de lograr la estabilidad, causalidad y frecuencia de corte de 3.3 kHz deseada en el filtro.

Se definen los parámetros del filtro, para el caso del diseño de filtros, el ancho de banda de transición de la ventana usada NO podrá ser mayor al ancho de banda de transición en radianes normalizado, por ende, se deberá cumplir la condición:

N > $(\alpha $f_s$) / (2*delta_{fm})$

Donde $\alpha->$ 4: "Ventanas rectangulares"; 8: "Ventanas Hann o Hamming"; 12: "Ventana Blackman".

In [ ]:
#CONSTANTS
fc = 3300.0 #Hz Cutoff frequency
delta_fm = 500.0 #Hz Roll-off
window = 'hanning' #rect,hanning,hamming,blackman

fc = fc + delta_fm / 2
Fs = Fs #44100 Hz Previously defined #Sampling frequency designed filter

def get_minimum_filter_order(window, fs, delta_fm):
    if window == 'rect':
        alpha = 4.0
    elif window == 'hanning' or window == 'hamming':
        alpha = 8.0
    elif window == 'blackman':
        alpha = 12.0
    return int(alpha*fs/(2*delta_fm)) + 1

N = get_minimum_filter_order(window, Fs, delta_fm) #Samples

print("Frecuencia de corte: %.2f Hz\nFrecuencia de muestreo: %.2f Hz\nAncho de banda de transici´on: %.2f Hz\nTipo de ventana: %s"%(fc, Fs, delta_fm, window))

Se definen parámetros secundarios para las bandas de paso, entre otros aspectos relacionados a incluir la primera muestra en el origen o graficar los puntos de cada muestra.

In [ ]:
#PARAMETERS
fmax = Fs / 2 #Hz Nyquist frequency
highpass = [fc, fmax] #Hz High Pass range frequencies

def isZeroFirst(N): return N % 2 == 1
#isZeroFirst = isZeroFirst(N) #If it is true, the first step starts with 0, otherwise starts with Fs/(2*N) instead of 0.
stem = True if N <= 100 else False #IF it is true, stem points will be included on graphs.

Se realiza definición de métodos que serán requeridos para el posterior análisis de la señal.

In [ ]:
#Secondary Methods

#Return the number of points required between 0 to π depending on the filter order is a par or odd number.
def get_alpha(N):
    alpha = int((N - 1) / 2) if isZeroFirst(N) else int(N / 2 - 1) #Alpha
    return alpha

#Get frequency interval (fk) related to x Axis of the frequency response of the filter.
def get_interval(alpha):
    bandwith = (Fs/N)
    sequence = np.arange(0,alpha + 1)
    interval = bandwith*sequence if isZeroFirst(N) else bandwith*(sequence+1/2) #Step vector
    return  interval

#Get the previous one as a centered interval.
def get_centered_interval(samples):
    bandwith = (Fs/N)
    
    alpha = (samples - 1) / 2 if isZeroFirst(N) else samples/2
    if isZeroFirst(N):
        sequence = np.arange(- alpha, alpha + 1)
    else:
        sequence = np.arange(- alpha, alpha)
        
    interval = bandwith*sequence if isZeroFirst(N) else bandwith*(sequence+1/2) #Step vector
    return  interval


El siguiente método finalmente se encargará de entregar la respuesta ideal deseada en el diseño del filtro pasa-altas. En el algoritmo se hizo énfasis en el aprovechamiento de las propiedades de simetría de la respuesta del filtro de acuerdo a las condiciones de fase lineal interpuestas al momento inicial del diseño de éste. De esta manera, tenemos que los coeficientes para el filtro de interés corresponden a: 

In [ ]:
#Return a frequency, and amplitude for Transfer functions of a high pass filter
def get_high_pass_transfer_function(fc, Fs, N):
    alpha = get_alpha(N)
    
    fk_right = get_interval(alpha) #Samples Interval
    Hk_right = (fk_right >= fc).astype(int) # Transfer functions high pass filter between 0 - π
    
    #Getting the left side of the functions according to its simetry (Due by linear fase relation).
    Hk_left = np.flip(Hk_right[1:]) if isZeroFirst(N) else np.concatenate((Hk_right[-1], np.flip(Hk_right[:-1])), axis=None) #np.flip(Hk_right[:-1]) # Transfer functions high pass filter between -π to 0

    fk = get_centered_interval(N) #Sample Interval
    
    Hk = np.concatenate((Hk_left, Hk_right), axis=None) # Transfer functions high pass filter between - π to π
    
    return fk, Hk

fk, Hk = get_high_pass_transfer_function(fc, Fs, N)

print("H(k): ", Hk)

Se grafica la respuesta al impulso y el espectro resultante en cada uno de los pasos metodológicos que requirió el diseño del filtro correspondiente.

A continuación se observa la respuesta deseada para el filtro FIR pasa-altas diseñado en el procedimiento anterior. Debido a las condiciones de linealidad en la fase del filtro, podemos observar la simetría presente entre [-π,0] y [0,π].

El caso actual tambi´en se ha adaptado para filtros de orden par o impar (es indiferente), con la única diferencia de que las muestras serán desplazadas $k -> k + \frac{1}{2}$.

In [ ]:
#Plot frequency response caused by a high pass filter
def plot_high_pass_filter_frequency_response(fk, Hk, stem = True):
    %matplotlib inline
    plt.figure(figsize=(20,8))
    plt.title("Respuesta deseada para filtro FIR pasa-altas", fontsize = 18)
    plt.xlabel('Frecuencia [Hz]',fontsize=18)
    plt.ylabel('Amplitud $|H_{w}|$',fontsize=18)
    plt.grid()
    if stem: plt.stem(fk, Hk)
    plt.plot(fk, Hk, label = 'Filtro pasa-altas\n$F_s$=%0.1f kHz\tOrden=%0.f'%(Fs*10**-3,N))
    plt.vlines(-fc, 0, 1, color='g', lw=1, linestyle='--')
    plt.vlines(fc, 0, 1, color='g', lw=1, linestyle='--', label='$f_c$=%0.1f kHz'%(fc*10**-3))
    plt.vlines(-fmax, 0, 1, color='r', lw=1, linestyle='--')
    plt.vlines(fmax, 0, 1, color='r', lw=1, linestyle='--', label='$f_{max}$=%0.1f kHz'%(fmax*10**-3))
    plt.legend()

plot_high_pass_filter_frequency_response(fk, Hk, stem = stem)


A continuación, al filtro pasa-altas se le aplicará una segunda transformación con la finalidad de generar una distorsión de forma cosenoidal para las frecuencias superiores a la frecuencia de corte (3.3 kHz). De esta manera, su respuesta en frecuencia continuará estando normalizada, pero sufrirá en ciertos tramos de frecuencia una atenuación de la misma.



In [ ]:
def apply_distortion(Hk):
    n = np.arange(0, len(Hk), 1)
    Hk = Hk*np.abs(np.cos(2*np.pi*n*fc/Fs))
    return Hk

Hk = apply_distortion(Hk)

print("H(k): ", Hk)
plot_high_pass_filter_frequency_response(fk, Hk, stem = stem)


Una vez se tiene bien definido el filtro deseado a diseñar en el dominio de la frecuencia, será necesario obtener entonces los coeficientes del filtro en el dominio del tiempo. 

(!) Recordar que todo filtro real sufre un retardo o desfase, por ende se debe desplazar una cantidad N/2 muestras a la derecha para su causalidad.

Se realiza simplificación del cálculo haciendo la fase del filtro lineal, es decir, para que su velocidad de grupo sea nula, de esta manera todas las componentes frecuenciales viajarán a la misma velocidad de grupo, o en otras palabras, estarán retardadas la misma cantidad de muestras.

$h(n) = \frac{1}{N} \sum_{k=0}^{N-1} H(k) e^{\frac{j2\pi kn}{N}}; k=0,1,2,...,N-1$

De esta manera, podemos reducir la expresión anterior a la siguiente expresión:

$\alpha = \begin{pmatrix}
\frac{N-1}{2}, N ~ par\\
\frac{N}{2}-1, N ~ impar
\end{pmatrix}$

$h(n) = \frac{1}{N} \left[  2\sum_{k=1}^{\alpha} H(k) cos({\frac{j2\pi k(n-\alpha)}{N})+ H(0)}\right]; k=0,1,2,...,\alpha;$

Por último, para obtener los coeficientes restantes del filtro, como consecuencia de la fase lineal mencionada previamente, resultaría la siguiente relación:

$h(n) = \pm h(N-n-1) = +h(N-n-1)$

De esta manera, tenemos finalmente que los coeficientes del filtro deseado a diseñar corresponden a:

In [ ]:
def get_high_pass_filter_coefficient(Hk, N):
    alpha = get_alpha(N)
    Hk_left = Hk[alpha:]
    hn_left = []
    for n in range(0, alpha + 1):
        summation = 0
        for k in range(1, alpha + 1):
            summation += Hk_left[k]*np.cos(2*np.pi*k*(n-alpha)/N) #Summation part
        hn_left.append((1/N)*(2*summation + Hk_left[0])) # Filter coefficients between 0 - π
    hn_right = np.flip(hn_left[:-1])
    hn = np.concatenate((hn_left, hn_right), axis=None) # Filter coefficients between - π to π
    return hn
        
hn = get_high_pass_filter_coefficient(Hk, N)


def print_filter_coefficient(hn, N):
    alpha = get_alpha(N)
    hn_left = hn[:alpha + 1]
    for i, coeff in enumerate(hn_left):
        if i != alpha: print("h'n(%i)=\t%.3f\t=h'n(%i)"%(i,coeff,N-i-1))
        else:  print("h'n(%i)=\t\t%.3f"%(i,coeff))

print_filter_coefficient(hn, N)

Al graficar los coeficientes en mención, obtendremos la respuesta del filtro al impulso unitario. Podemos ver de la gráfica siguiente que la respuesta al impulso unitario ahora se encuentra desfasada con respecto al origen, esto es a causa de la condición de fase lineal, que nos obliga a generar la simetría positiva requerida antes de desplazar ($\pi$ rad) los coeficientes de la salida del filtro que permita generar la condición de causalidad (requerida para su existencia en la vida real).

Luego debemos definir la ventana del filtro que hace referencia al efecto de Gibbs presente al realizar la convolución sobre un filtro real.

In [ ]:
#Avoid Gibb Effect using Hanning Window
def avoid_gibb_effect(M, filter_type):
    #M = M - 1
    n = np.arange(-M/2, M/2)
    if filter_type == 'hanning':
        win = 0.5*(1+np.cos(2*np.pi*n/(M-1)))
    elif filter_type == 'hamming':
        win = 0.54 + 0.46*np.cos(2*np.pi*n/(M-1))
    elif filter_type == 'blackman':
        win = 0.42 - 0.5*np.cos(2*np.pi*n/(M-1)) + 0.08*np.cos(4*np.pi*n/(M-1))
    else: raise Exception
    return win

hn_ = hn
hn = hn * avoid_gibb_effect(N, window)

def print_filter_coefficient(hn, N):
    alpha = get_alpha(N)
    hn_left = hn[:alpha + 1]
    for i, coeff in enumerate(hn_left):
        if i != alpha: print("hn(%i)=\t%.3f\t=hn(%i)"%(i,coeff,N-i-1))
        else:  print("hn(%i)=\t%.3f"%(i,coeff))

print_filter_coefficient(hn, N)

Por otra parte, el filtro es estable, ya que al ser un filtro FIR, no posee salidas que alimentan nuevamente la entrada del sistema (filtros IIR), y esto permite que no existan denominadores en la función de transferencia, es decir, que no existan polos (pero sí ceros, que no afectan la estabilidad del filtro, como veremos más adelante en el diagrama de polos y ceros del filtro actual).

In [ ]:
#Plot frequency response caused by a high pass filter
def plot_high_pass_filter_time_response(t, hn, stem = True, title = "Respuesta deseada para filtro FIR pasa-altas distorsionado"):
    %matplotlib inline
    plt.figure(figsize=(20,8))
    plt.title(title, fontsize = 18)
    plt.xlabel('Tiempo [s]',fontsize=18)
    plt.ylabel('$|H_{n}|$ Amplitud',fontsize=18)
    plt.grid()
    if stem: plt.stem(t, hn)
    plt.plot(t, hn, label = 'Filtro pasa-altas\n$f_c$=%0.1f kHz\tOrden=%0.f'%(fc*10**-3,N))
    plt.legend()

n = np.arange(0, N) if isZeroFirst(N) else np.arange(0, N-1)
t = (1/Fs)*n

plot_high_pass_filter_time_response(t, hn_, stem = stem, title = "Respuesta deseada para filtro FIR pasa-altas distorsionado SIN uso de ventana")

In [ ]:
n = np.arange(0, N) if isZeroFirst(N) else np.arange(0, N-1)
t = (1/Fs)*n

plot_high_pass_filter_time_response(t, hn, stem = stem, title = "Respuesta deseada para filtro FIR pasa-altas distorsionado CON uso de ventana")

Podemos observar la notable diferencia entre la respuesta del filtro al aplicar el truncamiento por ventana Hanning, podemos observar el suavizado de la función ideal del filtro (SINC), lo que nos ayuda con la reducción del nivel de Ripple en la salida del filtro.

Es bien conocido el hecho de que en la práctica nunca obtendremos un filtro con respuesta al impulso igual a la deseada en el efecto del diseño, esto es a causa de la cantidad infinita de muestras N (orden del filtro) requeridas al momento de realizar la transformada inversa discreta de fourier (IDFT).

Por otra parte, la respuesta en frecuencia real del filtro sólo nos podrá garantizar que en los puntos en los cuales se tomaron las muestras, se tendrá ese valor de amplitud o atenuación deseado, tal como se observa en la gráfica siguiente, donde podemos observar que en los puntos elegidos para las muestras, dicho nivel de amplificación es de la unidad (1) o cero (0) según el caso para H(k).

In [ ]:

def reflect_vector(vector, symmetry = True):
    if isZeroFirst(N):
        vector = np.concatenate((np.flip(vector[1:]), vector), axis=None) if symmetry else np.concatenate((-np.flip(vector[1:]), vector), axis=None)
    else:
        vector = np.concatenate((np.flip(vector), vector), axis=None) if symmetry else np.concatenate((-np.flip(vector), vector), axis=None)
    return vector

#Getting frequency response caused by the designed filter
def get_frequency_response(num, den = 1):
    f = get_interval(get_alpha(N))    
    f , hw = signal.freqz(num, den, fs = Fs)
    return f, hw

#Plot frequency response caused by the designed filter
def plot_filter_frequency_response(f, hw, stem):
    #Plotting between -π to π
    f = reflect_vector(f, symmetry = False)
    hw = reflect_vector(hw, symmetry = True)
    %matplotlib inline
    plt.figure(figsize=(20,8))
    plt.title("Respuesta diseñada para filtro FIR pasa-altas distorsionado", fontsize = 18)
    plt.xlabel('Frecuencia [Hz]',fontsize=18)
    plt.ylabel('Amplitud $|H_{w}|$',fontsize=18)
    plt.grid()
    plt.plot(f, abs(hw), label = 'Filtro pasa-altas distorsionado\n$F_s$=%0.1f kHz\tOrden=%0.f'%(Fs*10**-3,N))
    if stem: plt.stem(f, abs(hw))
    
    lims = [0, max(abs(hw))] if max(hn) > 0 else [0, min(abs(hw))]
    plt.vlines(-fc, lims[0],lims[1], color='g', lw=1, linestyle='--')
    plt.vlines(fc, lims[0],lims[1], color='g', lw=1, linestyle='--', label='$f_c$=%0.1f kHz'%(fc*10**-3))
    plt.vlines(-fmax, lims[0],lims[1], color='r', lw=1, linestyle='--')
    plt.vlines(fmax, lims[0],lims[1], color='r', lw=1, linestyle='--', label='$f_{max}$=%0.1f kHz'%(fmax*10**-3))
    plt.legend()

f, hw = get_frequency_response(hn)

plot_filter_frequency_response(f, hw, stem = False)
plt.show()
plot_high_pass_filter_frequency_response(fk, Hk, stem)

Al generar las gráficas de la respuesta en frecuencia de la fase del filtro diseñado, podemos observar que efectivamente se cumple con la propiedad mencionada al principio sobre la linealidad en la fase que permita garantizar que la velocidad de grupo de la señal sea la misma para cualquier componente frecuencia de la misma.

Del mismo modo, también se pudo observar una particularidad sobre la simetría en el cambio de fase del mismo $\Delta\theta(k)$, ya que parece corresponder a una simetría negativa con respecto a la salida en frecuencia de la fase de dicho filtro, tal que $\Delta\theta(k)=-\Delta\theta(-k)$, la cual a su vez se sigue cumpliendo para cualquier tipo de filtro de orden N (par o impar).

In [ ]:
#Getting phase response caused by the designed filter
def get_phase_response(num, den = 1):
    f = get_centered_interval(N)
    _ , hw = signal.freqz(num, den, worN = f, fs = Fs)#get_centered_interval(N, isZeroFirst))
    tetha = np.unwrap(np.angle(hw)) / np.pi
    return f, tetha

#Plot phase response caused by the designed filter
def plot_filter_phase_response(f, tetha, stem = True):
    #Plotting between -π to π
    tetha = tetha / np.pi
    %matplotlib inline
    plt.figure(figsize=(20,8))
    plt.title("Respuesta diseñada para filtro FIR pasa-altas", fontsize = 18)
    plt.xlabel('Frecuencia [Hz]',fontsize=18)
    plt.ylabel('Fase θ(f) [π Rad]',fontsize=18)
    plt.grid()
    if stem: plt.stem(f, tetha)
    plt.plot(f, tetha, label = 'Filtro pasa-altas\n$f_c$=%0.1f kHz\tOrden=%0.f'%(fc*10**-3,N))
    
    lims = [0, max(tetha)] if max(tetha) > 0 else [0, min(tetha)]
    plt.vlines(-fc, lims[0], lims[1], color='g', lw=1, linestyle='--')
    plt.vlines(fc, lims[0], lims[1], color='g', lw=1, linestyle='--', label='$f_c$=%0.1f kHz'%(fc*10**-3))
    plt.vlines(-fmax, lims[0], lims[1], color='r', lw=1, linestyle='--')
    plt.vlines(fmax, lims[0], lims[1], color='r', lw=1, linestyle='--', label='$f_{max}$=%0.1f kHz'%(fmax*10**-3))
    plt.legend()
    plt.legend()

f, tetha = get_phase_response(hn)

plot_filter_phase_response(f, tetha, stem = stem)

Un aspecto demasiado interesante en el diseño de filtros consiste en poder observar el diagrama de polos y ceros para analizar si se cumplen las condiciones para causalidad y estabilidad del filtro de interés, o en otras palabras, debemos lograr que los polos del filtro est´en contenidos en la circunferencia unitaria.

Para el filtro actual FIR, no nos debemos preocupar por cumplir la condición de estabilidad (dicha condición obliga a que los polos del filtro deben estar contenidos en la circunferencia unitaria |z|<1), ya que no existen polos en él.

In [ ]:
def plot_zeros_and_poles(num, den = 1, title = 'Diagrama de polos y ceros diseñado para filtro FIR pasa-altas\n$f_c$=%0.1f kHz\tOrden=%0.f'%(fc*10**-3,N)):
    w , hw = signal.freqz(num, den)
    sys1=signal.lti(num, den)
    ang=np.arange(0.0,2*np.pi,1/Fs)
    xp=np.cos(ang)
    yp=np.sin(ang)
    
    %matplotlib inline
    plt.figure(figsize=(20,8))
    plt.title(title, fontsize = 18)
    plt.xlabel('Re{z} [σ]',fontsize=18)
    plt.ylabel('Im{z} [jω]',fontsize=18)
    plt.plot(xp,yp,'--', label = '|z| = 1')
    plt.plot(sys1.zeros.real, sys1.zeros.imag, 'o', label = 'Ceros')
    plt.plot(sys1.poles.real, sys1.poles.imag, 'x', label = 'Polos')
    plt.legend()
    plt.grid()

plot_zeros_and_poles(hn)

Resulta demasiado interesante el fen´omeno particular que existe sobre la ubicaci´on de los ceros de dicha funci´on de transferencia, su notable simetr´ia con respecto a la circunferencia unitaria, su espaciado equidistante, adem´as de los cuatro ceros separados (dos contenidos en su interior, y dos excluidos a lo lejos).

Se desea corroborar la funcionalidad correcta del filtro, por lo cual se creará una señal artificial de la cual la frecuencia dependa del tiempo de forma polinómica con un orden arbitatrio (frequency_order) que permita generar una señal con componentes frecuenciales en todo el espectro desde $\frac{-F_s}{2}$ hasta  $\frac{+F_s}{2}$, se grafica el espectro de dicha señal, se reproduce, y se compara con la salida del filtro.

In [ ]:
from IPython.display import Audio # para escuchar la senal

#Sinusoidal signal const
frequency_order = 4
freq_signal = Fs
aliasing_factor = 0.99 #If factor > 1 then there will be aliasing on the output of the filter.

def make_polinomial_signal(frequency_order, freq_signal, freq_filter):
    last_t = (freq_filter/2)**(1/frequency_order)
    t = np.arange(0, aliasing_factor*last_t, 1.0/freq_signal) # Vector de tiempo
    f = t**frequency_order
    x = np.sin(2*np.pi*f*t)
    
    #Resampling the signal in order to make a signal with the same sample frequency of the designed filter.
    if freq_signal != freq_filter:
        signal_time = (len(x)/freq_signal)
        new_samples_number = int(signal_time*freq_filter)
        x = signal.resample(x, new_samples_number)
    return x

def deleting_residuals(t, x):
    #Deleting residual samples caused by the convolutions process.
    sizes = [len(t), len(x)]
    if sizes[1] - sizes[0] > 0: x = x[:-(sizes[1] - sizes[0])]
    elif sizes[0] - sizes[1] > 0: t = t[:-(sizes[0] - sizes[1])]
    return t, x

def plot_signal_spectrum(freq_SIGNAL, SIGNAL, freq_signal, lims, zoom = False, title = ''):
    plt.figure(figsize=(20,8))
    plt.grid()
    plt.plot(freq_SIGNAL,abs(SIGNAL)**2 / max(lims), label = 'Frecuencia de muestreo $F_s$=%0.1f kHz'%(freq_signal*10**-3))
    
    normalized_lims = [0.0 , 1.0] if not(zoom) else [0.0, float(min(lims)/max(lims))] #zoom config
    plt.ylim(normalized_lims)
    plt.xlabel('Frecuencia [Hz]',fontsize=18) # x Axis
    plt.ylabel('Amplitude $|C_k|^{2}$',fontsize=18) # Y Axis
    plt.title(title)
    plt.legend()
    plt.show()

x = make_polinomial_signal(frequency_order, freq_signal, Fs)
X = np.fft.fft(x) #Getting frequency response of the filtered signal.
freq_X = np.fft.fftfreq(X.size)*freq_signal

y = np.convolve(hn,x, mode="full")#Filtering de audio signal throught the designed filter.
Y = np.fft.fft(y) #Getting frequency response of the filtered signal.
freq_Y = np.fft.fftfreq(Y.size)*freq_signal

t=np.arange(0, float(len(x))/freq_signal, 1.0/freq_signal)
t, x = deleting_residuals(t, x)

plt.figure(figsize=(20,8))
plt.grid()
plt.plot(t,x, label = 'Frecuencia de Muestreo $F_s$=%0.1f kHz\nNúmero de muestras: %i'%(freq_signal*10**-3, len(x)))
plt.xlabel('Tiempo (s)',fontsize=18) # x Axis
plt.ylabel('Amplitud',fontsize=18) # Y Axis
plt.title('Señal Polinomial Original de orden: %i'%frequency_order)
last_t_fc= (fc)**(1/frequency_order)
last_t_fmax= (fmax)**(1/frequency_order)
plt.vlines(last_t_fc, -1,1, color='y', lw=2, linestyle='--', label='$f_c$=%0.1f kHz'%(fc*10**-3))
plt.vlines(last_t_fmax, -1,1, color='r', lw=2, linestyle='--', label='$f_{max}$=%0.1f kHz'%(fmax*10**-3))
plt.legend()
plt.show()

display(Audio(x, rate=freq_signal))

t = np.arange(0, float(float(len(y))/freq_signal), 1.0/freq_signal)
t, x = deleting_residuals(t, y)
    
plt.figure(figsize=(20,8))
plt.grid()
plt.plot(t,y, label = 'Frecuencia de Muestreo $F_s$=%0.1f kHz\nNúmero de muestras: %i'%(freq_signal*10**-3, len(y)))
plt.vlines(last_t_fc, -1,1, color='y', lw=2, linestyle='--', label='$f_c$=%0.1f kHz'%(fc*10**-3))
plt.vlines(last_t_fmax, -1,1, color='r', lw=2, linestyle='--', label='$f_{max}$=%0.1f kHz'%(fmax*10**-3))

plt.xlabel('Tiempo (s)',fontsize=18) # x Axis
plt.ylabel('Amplitud',fontsize=18) # Y AxisAxis
plt.title('Señal Polinomial Filtrada de orden: %i'%frequency_order)
plt.legend()
plt.show()

display(Audio(y, rate=freq_signal))

lims = [max(abs(X)**2), max(abs(Y)**2)]
plot_signal_spectrum(freq_X, X, freq_signal, lims, zoom = False, title = 'Densidad espectral de potencia señal polinómica original de orden: %i'%frequency_order)

#Fixing traslaping output due by FFT method
index = len(freq_Y)//2 if freq_signal >= Fs else len(freq_Y)//2 + 1
freq_Y = np.concatenate((freq_Y[index:], freq_Y[:index]), axis=None)
Y = np.concatenate((Y[index:], Y[:index]), axis=None)

plot_signal_spectrum(freq_Y, Y, freq_signal, lims, zoom = True, title = 'Densidad espectral de potencia señal polinómica filtrada de orden: %i'%frequency_order)


Se puede observar en la gráfica de la señal con respecto al tiempo que existen una líneas que representan el momento exacto en el tiempo en el cual la señal artificial alcanza la frecuencia de corte del filtro ($f_c$) y la frecuencia de Nyquist del mismo ($f_{max}$).

A continuación se realiza en análisis de la salida del filtro con el insumo de entrada, para esto es muy importante que todas las señales de entrada tengan la misma frecuencia de muestreo del filtro diseñado, de esta manera al generar la convolución, se podrá obtener la respuesta en el dominio del tiempo y la frecuencia acorde a los requerimientos planteados de manera inicial.

In [ ]:
from scipy.io.wavfile import read # libreria para lectura de archivos de audio
from scipy import signal

songs = []
freq_songs = []
time_songs = []
#songs, freq_songs, time_songs = load_audios(filenames)
songs, freq_songs = load_audios(filenames)

#Frequency domain FFT vectors
def make_fft_audios(songs, freq_songs):
    SONGS = []
    freq_SONGS = []
    songs_number = len (filenames) #number of songs to filter.
    for i in range(songs_number):
        SONGS.append(np.fft.fft(songs[i]))
        freq_SONGS.append(np.fft.fftfreq(SONGS[i].size)*freq_songs[i])
    return SONGS, freq_SONGS
    
SONGS = []
freq_SONGS = []

SONGS, freq_SONGS = make_fft_audios(songs, freq_songs)

Luego se grafica cada una de las dos señales de audio tanto en el dominio del tiempo como el dominio de la frecuencia que nos facilitará el análisis sobre los efectos causados por el filtro diseñado.

In [ ]:
#Reproduction song objects vector
push_songs=[]
for i, filename in enumerate(filenames):
    print("\n\n\n\n\n________________________________________________________________________________________________________")
    print("Audio %i; Filename:"%i, filename)
    push_songs.append(Audio(songs[i], rate=freq_songs[i]))
    
    y = np.convolve(hn,songs[i], mode="full")#Filtering de audio signal throught the designed filter.

    Y = np.fft.fft(y) #Getting frequency response of the filtered signal.
    freq_Y = np.fft.fftfreq(Y.size)*freq_songs[i]
    
    graph_audio(songs[i], freq_songs[i])
    display(push_songs[i])
    
    graph_audio(y, freq_songs[i], title = "Señal de audio de salida en filtro FIR pasa-altas")
    display(Audio(y, rate=freq_songs[i]))
    
    lims = [max(abs(SONGS[i])**2),max(abs(Y)**2)]
    plot_signal_spectrum(freq_SONGS[i], SONGS[i], freq_songs[i], lims, zoom = False)
    plot_signal_spectrum(freq_Y, Y, freq_songs[i], lims, zoom = True)

del (i)
del (filename)
del (y)
del (Y)
del (freq_Y)

Podemos observar el correcto funcionamiento del filtro, al efectuar el filtrado de las frecuencias menores a 3.3 kHz, y la distorsión cosenoidal aplicada en la frecuencia de paso del filtro pasa-altas en mención.

#### Filtro IIR A Partir de Filtro Analógico Butterworth y Aproximación de Derivadas

Se realiza el diseño de un filtro IIR pasa-bajas usando el método de aproximación de derivadas y un filtro butterworth de primer orden.

Recordar que para un filtro analógico de primer orden, la frecuencia angular de corte tendrá un nivel de atenuación exactamente de -3 dB al deseado. Para este caso vemos que la relación $w_c = 2 \pi \frac {fc} {fs}$, entonces si deseamos un valor en frecuencia fc igual al anterior caso de análisis, entonces la frecuencia de muestreo dependerá de: $f_s=\frac{2 \pi fc}{w_c}$, con $w_c -> cte$

In [ ]:
#LIBRARIES
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal

In [ ]:
#VARIABLES
Nfft = N
T = 0.00001

#DESIGN CONSTS
Adb = 3 # Atenuation dB
fc = fc # Hz Previous cut-off frequency for FIR filter
fs = Fs

#PARAMETERS
wc =2*np.pi*(fc) / fs
fmax = fs / 2

Se genera de forma programática la función de transferencia del filtro analógico generado de acuerdo a la especificaciones solicitadas del mismo. 

In [ ]:
b, a = signal.butter(1, [2*np.pi*fc], 'lowpass', analog = True, output = 'ba', fs = None) #Analogic low pass filter
b, a

Se observa que para las especificaciones mencionadas, la ecuación de transferencia del filtro análogo será:

$H(s)=\frac{2 \pi 3550}{s+2 \pi 3550}$

$w_c = 0.160997732 \pi [Hz]$

Se grafica la respuesta al impulso y el espectro resultante en cada uno de los pasos metodológicos que requirió el diseño del filtro correspondiente.

Al realizar la gráfica de la respuesta del filtro Butterworth analógico de primer orden en el espectro, podemos ver

In [ ]:
def plot_signal_spectrum(freq_SIGNAL, SIGNAL, freq_signal, vlines, title, ylabel='Amplitude $|C_k|^{2}$'):
    plt.figure(figsize=(20,8))
    plt.grid()
    plt.plot(freq_SIGNAL,SIGNAL, label = 'Frecuencia de muestreo $F_s$=%0.1f kHz'%(freq_signal*10**-3))
    plt.stem(freq_SIGNAL,SIGNAL)
    
    plt.vlines(vlines[0], 0,1, color='g', lw=1, linestyle='--', label='$f_c$=%0.1f kHz (AdB = -18dB)'%(fc*10**-3))
    plt.vlines(vlines[1], 0,1, color='r', lw=1, linestyle='--', label='$f_{max}$=%0.1f kHz'%(fmax*10**-3))

    plt.xlabel('Frecuencia [Hz]',fontsize=18) # x Axis
    plt.ylabel(ylabel,fontsize=18) # Y Axis
    plt.title(title, fontsize=18)
    plt.legend()
    plt.show()
    
    
vlines = [fc, fmax]#[2*np.pi*fc, 2*np.pi*fmax]
w = np.arange(0, 2*np.pi*fmax, 2*np.pi*fmax/Nfft)
w, h = signal.freqs(b,a, worN = w)
plot_signal_spectrum(w / (2*np.pi), np.abs(h), fs, vlines, title='Respuesta en frecuencia filtro Butterworth analógico de primer orden', ylabel = 'Amplitude $|H(w)|$')

Luego, para transformar el filtro analógico en mención, a un filtro digital recursivo de primer orden (IIR), debemos hacer:

$s^{k} = (\frac{1-z^{-1}}{T})^{k} => s = \frac{1-z^{-1}}{T}, k=1 $

De esta manera, tenemos que la función de transferencia en el plano Z, será: 

$H(z) = \frac{2 \pi f_c Tz}{(1+2 \pi f_c T)z-1} = \frac{2 \pi 3550 Tz}{(1+2 \pi 3550 T)z-1} $

Para volver causal el filtro IIR, es necesario que los polos del filtro estén contenidos en la circunferencia unidad $|z|<1 => z = |\frac{1}{1+2 \pi f_cT}| < 1$, por ende, al resolver la inecuación, podemos observar que para cualquier $ T < \frac{-2}{2 \pi f_c}$ or $T > 0$, se puede tener un filtro causal.

Si deseáramos analizar en el dominio de la frecuencia, podríamos aplicar la transformada de Fourier haciendo $z=e^{\frac{j2 \pi k n}{N}}$ con la finalidad de convertir la función de transferencia en el plazo Z, en la función de transferencia $H(k)$ discreta deseada.


$H(k) = \frac{2 \pi 3550 Te^{\frac{j2 \pi k n}{N}}}{(1+2 \pi 3550 T)e^{\frac{j2 \pi k n}{N}}-1} $

Por ejemplo, si graficamos generamos 8 muestras de la transformada, usando el T mencionado anteriormente, tenemos:

In [ ]:
def den(N, T):
    n = np.arange(N)
    k= np.arange(N)
    den = ((1+2*np.pi*3550*T)*np.exp(1j* 2*np.pi/N *k*n) -1)
    return den

def Hk(N, T):
    k= np.arange(N)
    n = np.arange(N)
    Hk = 2*np.pi*3550*T*np.exp(1j* 2*np.pi/N *k*n) / ((1+2*np.pi*3550*T)*np.exp(1j* 2*np.pi/N *k*n) -1)
    return Hk
#e.g.
Hk(8, T)

Ahora se realizará la gráfica respectiva para el diagrama de polos y ceros del nuevo filtro generado (IIR), del cual se pudo observar que tantos sus ceros como sus polos se encuentran contenidos en la circunferencia unidad, lo que corrobora el hecho de que los coeficientes corresponden a un filtro causal y estable.

In [ ]:
def plot_zeros_and_poles(num, den = 1, title = 'Diagrama de polos y ceros diseñado para filtro FIR pasa-altas\n$f_c$=%0.1f kHz\tOrden=%0.f'%(fc*10**-3,N)):
    w , hw = signal.freqz(num, den)
    sys1=signal.lti(num, den)
    ang=np.arange(0.0,2*np.pi,1/Fs)
    xp=np.cos(ang)
    yp=np.sin(ang)
    
    %matplotlib inline
    plt.figure(figsize=(20,8))
    plt.title(title, fontsize = 18)
    plt.xlabel('Re{z} [σ]',fontsize=18)
    plt.ylabel('Im{z} [jω]',fontsize=18)
    plt.plot(xp,yp,'--', label = '|z| = 1')
    plt.plot(sys1.zeros.real, sys1.zeros.imag, 'o', label = 'Ceros')
    plt.plot(sys1.poles.real, sys1.poles.imag, 'x', label = 'Polos')
    plt.legend()
    plt.grid()

plot_zeros_and_poles(hn)

b11 = [2*np.pi*4750*T, 0]
a11 = [1 + 2*np.pi*4750*T, -1]
plot_zeros_and_poles(b11,a11, title='Diagrama de polos y ceros diseñado para filtro IIR pasa-bajas\n$f_c$=%0.1f kHz a través de filtro Butterworth analógico de Orden=1'%(fc*10**-3))
print ("numerador: ",b11)
print("denominador: ",a11)

Como ya pudimos corroborar que el filtro puede ser diseñado en la práctica, vamos a efectuar la respuesta del filtro al impulso unitario, de esta manera podremos analizar el comportamiento del filtro para la frecuencia de corte asignada inicialmente.

In [ ]:
import scipy.signal as signal
from pylab import repeat
import matplotlib.pyplot as plt
import numpy as np
    
def impz(b,a):
    l = Nfft
    impulse = repeat(0.,l); impulse[0] =1.
    x = np.arange(0,l)
    response = signal.lfilter(b,a,impulse)
    t = (1/fs)*x
    return t, response

def graph_impz(t, response):
    plt.figure(figsize=(20,8))
    plt.grid()
    plt.plot(t, response, label = 'Frecuencia de muestreo $F_s$=%0.1f kHz'%(fs*10**-3))
    plt.stem(t, response)
    plt.xlabel('Tiempo [s]',fontsize=18) # x Axis
    plt.ylabel('Amplitud',fontsize=18) # Y Axis
    plt.title('Respuesta al impulso para filtro digital IIR pasa bajas primer orden', fontsize=18)
    plt.legend()
    plt.show()

t, hn = impz(b11,a11)

graph_impz(t, hn)


Al observar la respuesta al impulso para dicho filtro resultante (digital), podemos observar que la función SINC ha sido atenuada abruptamente, sin embargo no podemos obtener mucha informaci´on desde el dominio del tiempo.

Si realizamos una comparación entre el filtro análogo usado como insumo de diseño, y el filtro digital resultante, podemos observar ciertas diferencias notorias sobre la frecuencia de corte del mismo, como lo fue un desplazamiento "relativo" de esta (perspectiva negativa), o por otra parte, una atenuación abrupta (perspectiva positiva) que permitió maximizar la eficiencia del filtro (Butterworth primer orden análogo), con respecto a la anterior (filtro FIR digital)

In [ ]:

def plot_frequency_response(freq_SIGNAL, SIGNAL, freq_signal, lims, zoom = False, title = ''):
    plt.figure(figsize=(20,8))
    plt.grid()
    plt.plot(freq_SIGNAL,abs(SIGNAL) / max(lims), label = 'Frecuencia de muestreo $F_s$=%0.1f kHz'%(fs*10**-3))
    plt.stem(freq_SIGNAL,abs(SIGNAL) / max(lims))
    plt.axvline(fc, color='g', lw=1, linestyle='--', label='$f_c$=%0.1f kHz (AdB = -3dB; |H(w)| = 0,708)'%(fc*10**-3))
    plt.axvline(fmax, color='r', lw=1, linestyle='--', label='$f_{max}$=%0.1f kHz'%(fmax*10**-3))
    
    normalized_lims = [0.0 , 1.0] if not(zoom) else [0.0, float(min(lims)/max(lims))] #zoom config
    plt.ylim(normalized_lims)
    plt.xlim([0.0 , fs/2])
    plt.xlabel('Frecuencia [Hz]',fontsize=18) # x Axis
    plt.ylabel('Amplitud $|H(w)|$',fontsize=18) # Y Axis
    plt.title('Espectro de la respuesta al impulso para filtro digital IIR pasa bajas primer orden', fontsize=18)
    plt.legend()
    plt.show() #Respuesta en frecuencia filtro Butterworth DIGITAL pasa-bajas de primer orden
    
Hn = np.fft.fft(hn) #Getting frequency response of the filtered signal.
freq_Hn = np.fft.fftfreq(Hn.size)*fs

plot_frequency_response(freq_Hn, Hn, fs, lims=[0,max(abs(Hn))])

def plot_signal_spectrum(freq_SIGNAL, SIGNAL, freq_signal, vlines, title, ylabel='Amplitude $|C_k|^{2}$'):
    plt.figure(figsize=(20,8))
    plt.grid()
    plt.plot(freq_SIGNAL,SIGNAL, label = 'Frecuencia de muestreo $F_s$=%0.1f kHz'%(freq_signal*10**-3))
    plt.stem(freq_SIGNAL,SIGNAL)
    
    plt.vlines(vlines[0], 0,1, color='g', lw=1, linestyle='--', label='$f_c$=%0.1f kHz (AdB = -3dB; |H(w)| = 0,708)'%(fc*10**-3))
    plt.vlines(vlines[1], 0,1, color='r', lw=1, linestyle='--', label='$f_{max}$=%0.1f kHz'%(fmax*10**-3))

    plt.xlabel('Frecuencia [Hz]',fontsize=18) # x Axis
    plt.ylabel(ylabel,fontsize=18) # Y Axis
    plt.title(title, fontsize=18)
    plt.legend()
    plt.show()
    
    
vlines = [fc, fmax]#[2*np.pi*fc, 2*np.pi*fmax]
w = np.arange(0, 2*np.pi*fmax, 2*np.pi*fmax/Nfft)
w, h = signal.freqs(b,a, worN = w)
plot_signal_spectrum(w / (2*np.pi), np.abs(h), fs, vlines, title='Respuesta en frecuencia filtro Butterworth analógico de primer orden', ylabel = 'Amplitude $|H(w)|$')

De esta manera como se conoce la respuesta al impulso del filtro en mención tanto en el tiempo como en la frecuencia, se podrá calcular la salida de cualquier señal de entrada a dicho filtro a través de operaciones en el tiempo (convolución) o en la frecuencia, por lo cual no existirá ninguna limitante para el análisis del filtrado de los tres audios implementados en el análisis del caso anterior.

Es bien conocido el hecho de que en la práctica nunca obtendremos un filtro con respuesta al impulso igual a la deseada en el efecto del diseño, esto es a causa de la cantidad infinita de muestras N (orden del filtro) requeridas al momento de realizar la transformada inversa discreta de fourier (IDFT).

Al iterar varias veces el período arbitrario T, se pudo observar que al iterar con el período T = 0.00001, la función de transferencia resultante $H(z)=\frac{0.2230553078z}{(1.2230553078)z-1}$ parece sobrepasar por mucho a los requisitos requeridos de atenuación (-3dB) original del filtro an´alogo.

In [ ]:

wb, Hb = signal.freqz(b11, a11, worN = Nfft)
wb = np.arange(0, fmax, fmax/Nfft)
plt.figure(figsize=(20,8))
plt.title('Respuesta en frecuencia filtro digital IIR a partir de filtro Butterworth analógico de primer orden',fontsize=18)
plt.plot(wb,np.abs(Hb),label='Frecuencia de muestreo $F_s$=%0.1f kHz'%(fs*10**-3))
plt.stem(wb,np.abs(Hb))
plt.axvline(fc, color='g', lw=1, linestyle='--', label='$f_c$=%0.1f kHz (AdB = -3dB; |C_w| = 0,708)'%(fc*10**-3))
plt.axvline(fmax, color='r', lw=1, linestyle='--', label='$f_{max}$=%0.1f kHz'%(fmax*10**-3))
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Amplitude $|H(w)|$')
plt.ylim([0,1])
plt.legend()
plt.grid()
plt.show()

w = np.arange(0, 2*np.pi*fmax, 2*np.pi*fmax/Nfft)
w, h = signal.freqs(b,a, worN = w)
plot_signal_spectrum(w / (2*np.pi) , np.abs(h), fs, vlines, title='Respuesta en frecuencia filtro Butterworth analógico de primer orden')

Con el valor asignado para el período T = 0.00001, podemos observar que el filtro IIR en mención puede lograr atenuarse en alrededor de dos veces más que en el caso del filtro Butterworth analógico de primer orden. Además se puede observar que las componentes frecuenciales que se encuentran dentro de la banda de paso (antes de la frecuencia de corte) también sufrieron el fenómeno de atenuación, en el cual las componentes en $\frac{f_c}{2}$ se atenuan en un 40% del valor original. 

Lo anterior genera una perspectiva relativa en la cual parece que la variable T hubiera generado un efecto relativo en el desplazamiento de la frecuencia de corte al momento de diseñarse el filtro en la realidad.

Sin embargo, el período asignado fue obtenido al iterar estratégicamente para obtener un filtro arbitrario, es decir, que podemos con la variable T, generar infinitas respuestas en frecuencia.

NO fue posible lograr obtener una relación que permitiera obtener el T deseado para una frecuencia de corte y atenuación deseada de forma teórica, la única posibilidad para nuestro caso es iterar de forma secuencial hasta lograr obtener los parámetros deseados de Ripple y Roll-off.



Al realizar un análisis sobre la fase del filtro IIR en mención, podemos observar que efectivamente se dejó de cumplir con la propiedad de linealidad de fase mencionada en el sistema anterior (Filtro pasa-altas FIR). Esto es a causa de que al ser un filtro recursivo, sus entradas están re-alimentadas por la salida, y dependerán de estados iniciales (memoria), esto implica la consecuencia en que la fase no sea lineal.


In [ ]:
#Getting phase response caused by the designed filter
def get_IIR_phase_response(num, den, N):
    f = np.arange(0, fmax, Nfft/fs)
    _ , hw = signal.freqz(num, den, worN = f, fs = Fs)#get_centered_interval(N, isZeroFirst))
    tetha = np.unwrap(np.angle(hw)) / np.pi
    return f, tetha

#Plot phase response caused by the designed filter
def plot_IIR_filter_phase_response(f, tetha, stem = True):
    tetha = tetha / np.pi
    %matplotlib inline
    plt.figure(figsize=(20,8))
    plt.title("Respuesta en fase diseñada para filtro IIR digital pasa-bajas orden 1", fontsize = 18)
    plt.xlabel('Frecuencia [Hz]',fontsize=18)
    plt.ylabel('Fase θ(f) [π Rad]',fontsize=18)
    plt.grid()
    if stem: plt.stem(f, tetha)
    plt.plot(f, tetha, label = 'Filtro IIR digital pasa-bajas\n$f_c$=%0.1f kHz\n$f_s$=%0.1f kHz'%(fc*10**-3,fs*10**-3))
    plt.legend()
    plt.legend()

f, tetha = get_IIR_phase_response(b11, a11, N = Nfft)
plot_IIR_filter_phase_response(f, tetha, stem = False)


Sin embargo, de acuerdo a la gráfica de fase observada, se observan ciertos rangos o porciones de frecuencia, en las cuales la fase presenta propiedades de linealidad y se podría aproximar a una recta. Lo interesante es que en las vecindades del punto de la frecuencia de corte, dicha fase NO es lineal, pero a medida que nos alejamos de dicho punto, se observa dos rectas que permiten modelar la fase lineal. 

Lo anterior, en otras palabras significa que las componentes frecuencias de la señal de entrada que est´an cerca a la frecuencia de corte del filtro, siempre sufrir´an desfases en los cuales cada componente (o armonico) en frecuencia se desfasar´a una cantidad distinta de muestras (dependiento de la derivada del cambio de fase); por otra parte, las componentes frecuenciales lejanas a dicha frecuencia de corte sufrir´an un desplazamiento lineal, lo que permitir´a que su velocidad de grupo sea la misma, y su retardo sea en la misma cantidad de muestras, por lo que corrobora que a la salida NO se haya observado ninguna distorsi´on.

Igualmente que el caso anterior, se desea corroborar la funcionalidad correcta del filtro, por lo cual se creará una señal artificial de la cual la frecuencia dependa del tiempo de forma polinómica con un orden arbitatrio (frequency_order) que permita generar una señal con componentes frecuenciales en todo el espectro desde $\frac{-F_s}{2}$ hasta  $\frac{+F_s}{2}$, se grafica el espectro de dicha señal, se reproduce, y se compara con la salida del filtro.

In [ ]:
from IPython.display import Audio # para escuchar la senal

#Sinusoidal signal const
frequency_order = 4
freq_signal = fs
aliasing_factor = 1.1 # If the factor > 1, there will be aliasing for the output of the digital filter.

def impz(b,a):
    l = Nfft
    impulse = repeat(0.,l); impulse[0] =1.
    x = np.arange(0,l)
    response = signal.lfilter(b,a,impulse)
    t = (1/fs)*x
    return t, response

def make_polinomial_signal(frequency_order, freq_signal, freq_filter):
    last_t = (freq_filter/2)**(1/frequency_order)
    t = np.arange(0, aliasing_factor*last_t, 1.0/freq_signal) # Vector de tiempo
    f = t**frequency_order
    x = np.sin(2*np.pi*f*t)
    
    #Resampling the signal in order to make a signal with the same sample frequency of the designed filter.
    if freq_signal != freq_filter:
        signal_time = (len(x)/freq_signal)
        new_samples_number = int(signal_time*freq_filter)
        x = signal.resample(x, new_samples_number)
    return x

def deleting_residuals(t, x):
    #Deleting residual samples caused by the convolutions process.
    sizes = [len(t), len(x)]
    if sizes[1] - sizes[0] > 0: x = x[:-(sizes[1] - sizes[0])]
    elif sizes[0] - sizes[1] > 0: t = t[:-(sizes[0] - sizes[1])]
    return t, x

def plot_signal_spectrum(freq_SIGNAL, SIGNAL, freq_signal, lims, zoom = False, title = ''):
    plt.figure(figsize=(20,8))
    plt.grid()
    plt.plot(freq_SIGNAL,abs(SIGNAL)**2 / max(lims), label = 'Frecuencia de muestreo $F_s$=%0.1f kHz'%(freq_signal*10**-3))
    
    normalized_lims = [0.0 , 1.0] if not(zoom) else [0.0, float(min(lims)/max(lims))] #zoom config
    plt.ylim(normalized_lims)
    plt.xlabel('Frecuencia [Hz]',fontsize=18) # x Axis
    plt.ylabel('Amplitude $|H(w)|^{2}$',fontsize=18) # Y Axis
    plt.title(title)
    plt.legend()
    plt.show()
    
x = make_polinomial_signal(frequency_order, freq_signal, fs)
X = np.fft.fft(x) #Getting frequency response of the filtered signal.
freq_X = np.fft.fftfreq(X.size)*freq_signal

t, hn = impz(b11,a11)
y = np.convolve(hn,x, mode="full")#Filtering de audio signal throught the designed filter.
Y = np.fft.fft(y) #Getting frequency response of the filtered signal.
freq_Y = np.fft.fftfreq(Y.size)*freq_signal

t=np.arange(0, float(len(x))/freq_signal, 1.0/freq_signal)
t, x = deleting_residuals(t, x)

plt.figure(figsize=(20,8))
plt.grid()
plt.plot(t,x, label = 'Frecuencia de Muestreo $F_s$=%0.1f kHz\nNúmero de muestras: %i'%(freq_signal*10**-3, len(x)))
plt.xlabel('Tiempo (s)',fontsize=18) # x Axis
plt.ylabel('Amplitud',fontsize=18) # Y Axis
plt.title('Señal Polinomial Original de orden: %i'%frequency_order)
last_t_fc= (fc)**(1/frequency_order)
last_t_fmax= (fmax)**(1/frequency_order)
plt.vlines(last_t_fc, -1,1, color='y', lw=2, linestyle='--', label='$f_c$=%0.1f kHz'%(fc*10**-3))
plt.vlines(last_t_fmax, -1,1, color='r', lw=2, linestyle='--', label='$f_{max}$=%0.1f kHz'%(fmax*10**-3))
plt.legend()
plt.show()

display(Audio(x, rate=freq_signal))

t = np.arange(0, float(float(len(y))/freq_signal), 1.0/freq_signal)
t, x = deleting_residuals(t, y)
    
plt.figure(figsize=(20,8))
plt.grid()
plt.plot(t,y, label = 'Frecuencia de Muestreo $F_s$=%0.1f kHz\nNúmero de muestras: %i'%(freq_signal*10**-3, len(y)))
plt.vlines(last_t_fc, -1,1, color='y', lw=2, linestyle='--', label='$f_c$=%0.1f kHz'%(fc*10**-3))
plt.vlines(last_t_fmax, -1,1, color='r', lw=2, linestyle='--', label='$f_{max}$=%0.1f kHz'%(fmax*10**-3))

plt.xlabel('Tiempo (s)',fontsize=18) # x Axis
plt.ylabel('Amplitud',fontsize=18) # Y AxisAxis
plt.title('Señal Polinomial de orden: %i a través de filtro IIR digital de orden 1'%frequency_order)
plt.legend()
plt.show()

display(Audio(y, rate=freq_signal))

lims = [max(abs(X)**2), max(abs(Y)**2)]
plot_signal_spectrum(freq_X, X, freq_signal, lims, zoom = False, title = 'Espectro de señal polinómica original de orden: %i'%frequency_order)

#Fixing traslaping output due by FFT method
index = len(freq_Y)//2 if freq_signal >= fs else len(freq_Y)//2 + 1
freq_Y = np.concatenate((freq_Y[index:], freq_Y[:index]), axis=None)
Y = np.concatenate((Y[index:], Y[:index]), axis=None)

plot_signal_spectrum(freq_Y, Y, freq_signal, lims, zoom = True, title = 'Espectro de señal polinómica filtrada de orden: %i a través de filtro IIR digital de orden 1'%frequency_order)


Para este caso podemos observar nuevamente que el filtro diseñado parece atenuar las componentes frecuenciales luego de los 3.55kHz cercano a la frecuencia de corte requerida en el diseño. Se puede observar en la gráfica de la señal con respecto al tiempo que existen una líneas que representan el momento exacto en el tiempo en el cual la señal artificial alcanza la frecuencia de corte del filtro ( 𝑓𝑐 ) y la frecuencia de Nyquist del mismo ( 𝑓𝑚𝑎𝑥 ).

A continuación se realiza nuevamente la carga de dos audios.


In [ ]:
songs = []
freq_songs = []
time_songs = []
songs, freq_songs = load_audios(filenames)

#Frequency domain FFT vectors
def make_fft_audios(songs, freq_songs):
    SONGS = []
    freq_SONGS = []
    songs_number = len (filenames) #number of songs to filter.
    for i in range(songs_number):
        SONGS.append(np.fft.fft(songs[i]))
        freq_SONGS.append(np.fft.fftfreq(SONGS[i].size)*freq_songs[i])
    return SONGS, freq_SONGS
    
SONGS = []
freq_SONGS = []

SONGS, freq_SONGS = make_fft_audios(songs, freq_songs)


Ahora al generar la convolución, se pudo obtener una respuesta en el dominio del tiempo y la frecuencia que parece cumplir con los requerimientos planteados inicialmente en la práctica, de igual manera que en el caso anterior, dado el orden del filtro en mención resulta demasiado difícil observar diferencias notables para las señales de audio, por lo cual era necesario realizar el análisis del filtro en mención usando la señal artificial anterior que pudiera aportar mayor cantidad de componentes frecuenciales en todo el espectro y no sólo en las zonas de baja frecuencia como suele ocurrir en la realidad (de acuerdo al espectro de la voz y el oído humano).

In [ ]:
#Reproduction song objects vector
push_songs=[]

def impz(b,a):
    l = Nfft
    impulse = repeat(0.,l); impulse[0] =1.
    x = np.arange(0,l)
    response = signal.lfilter(b,a,impulse)
    t = (1/fs)*x
    return t, response

t, hn = impz(b11,a11)


for i, filename in enumerate(filenames):
    print("\n______________________________\n")
    print("Audio %i; Filename:"%i, filename)
    push_songs.append(Audio(songs[i], rate=freq_songs[i]))
    
    y = np.convolve(hn,songs[i], mode="full")#Filtering the audio signal throught the designed filter.
    
    Y = np.fft.fft(y) #Getting frequency response of the filtered signal.
    freq_Y = np.fft.fftfreq(Y.size)*freq_songs[i]
    
    graph_audio(songs[i], freq_songs[i])
    display(push_songs[i])
    
    graph_audio(y, freq_songs[i], title = "Señal de audio de salida en filtro IIR pasa-bajas orden 1")
    display(Audio(y, rate=freq_songs[i]))
    
    lims = [max(abs(SONGS[i])**2),max(abs(Y)**2)]
    newrange = np.arange(0, len(freq_SONGS[i]), 1)
    plot_signal_spectrum(freq_SONGS[i], SONGS[i], freq_songs[i], lims, title ='')
    plot_signal_spectrum(freq_Y, Y, freq_songs[i], lims, zoom = True, title ='')

del (i)
del (filename)
del (y)
del (Y)
del (freq_Y)

Finalmente, para el audio 0 no se pudo observar gran diferencia, a causa que sus sonidos poseen mayor energía en las zonas de frecuencias graves; por otra parte, para el audio 1 se puede observar la diferencia entre la entrada y salida del audio a través del filtro IIR, en la cual se han atenuado de forma leve las componentes de frecuencia altas del sonido, a pesar de tener un orden 1, se pudo observar el cambio, dado que el sonido es un Beat de electrónica entre 113 y 120 BPM, que suelen posee componentes agudas en contraste a los sonidos que encontramos en la naturaleza.

#### Conclusiones sobre filtros FIR e IIR

Como conclusión final, se pudo observar la facilidad en el diseño del filtro FIR a partir del diseño de su espectro en el dominio de la frecuencia y a través del uso de la transformada inversa (IDFT) para poder reconstruirla en el dominio del tiempo y así hallar los coeficientes.

Del mismo modo aplica para el caso del filtro IIR, en este caso resulta interesante la flexibilidad en el diseño que aporta el uso de la aproximación de derivadas que permite transformar un sistema complejo en otro más simple, que puede ser resuelto por métodos numéricos.

Se pudo observar mayor eficiencia en el diseño del filtro IIR, en el cual con ayuda de los grados de libertad (Período T) se pudo asignar por iteración los requerimientos aproximadamente deseados por el filtro con respecto a la frecuencia de corte y el nivel de roll-off, lo cual nos permite seguir corroborando el hecho de que resulta más eficiente su implementación con respecto a un filtro FIR.

En otras palabras, si los requerimientos de diseño para un filtro FIR requieren que éste sea de un orden elevado (más complejo y costoso), será más óptimo en relación costo-beneficio implementar un filtro IIR el cual podrá cumplir los requerimientos de diseño iniciales, pero esta vez con un orden mucho menor (más simple y barato).